In [ ]:
import sys
import datetime as dt
from dateutil.relativedelta import relativedelta
from pathlib import Path

import pandas as pd

from arcgis.gis import GIS

### Connect to the GIS as an administrator

In [ ]:
gis = GIS(profile="your_gis_admin", verify_cert=False)

## Helper functions

### Get valid `user type` names for a spreadsheet column or the `user_type` parameter

In [ ]:
def get_valid_user_types():
    return (lic['name'] for lic in gis.users.license_types)

In [ ]:
for ut in get_valid_user_types():
    print(ut)

### Get the number of user types already consumed for a specific user type

In [ ]:
def get_user_type_count(user_type):
    user_type_mapping = zip([lic['name'] for lic in gis.users.license_types],
                            [lic['id'] for lic in gis.users.license_types])
    user_type = [utm[1] for utm in user_type_mapping if utm[0] == user_type][0]
    ut_counts_df = gis.users.counts(type="user_type")
    if user_type in list(ut_counts_df["key"]):
        return ut_counts_df.loc[ut_counts_df["key"] == user_type]["count"].iloc[0]
    else:
        return 0

In [ ]:
get_user_type_count("GIS Professional Advanced")

### Get the maximum number of user types available for a specific user type

In [ ]:
def get_user_type_max(user_type):
    return next(lic['maxUsers'] 
                for lic in gis.users.license_types 
                if lic['name'] == user_type)

In [ ]:
get_user_type_max("GIS Professional Advanced")

### Function to check whether a user type license remains for a specific user type

In [ ]:
def is_user_type_available(user_type):
    if get_user_type_max(user_type) - get_user_type_count(user_type) > 0:
        return True
    else:
        return False

In [ ]:
is_user_type_available("GIS Professional Advanced")

### Get a reference to a file with required information to create users

In [ ]:
intern_data_path = Path(r"path_to_directory")

In [ ]:
intern_file = next(intern_data_path.glob("criteria_to_find_file"))

In [ ]:
intern_data = pd.read_csv(intern_file)
intern_data

### Change an `Administrator` role values since those cannot be assigned automatically

In [ ]:
for i in list(intern_data[intern_data.Role == "Administrator"].index):
    intern_data.at[i, "Role"] = "Publisher"

### Add text to ensure uniqueness of username across ArcGIS Online

In [ ]:
# optional
intern_data = intern_data.apply(lambda un:un+"unique_str" if un.name == "Username" else un)

In [ ]:
# optional
intern_data = intern_data.apply(lambda pw:pw+"unique_str" if pw.name == "Password" else pw)

In [ ]:
intern_data

### Loop through the file to create users

* use a `try/except` clause to catch for errors
* use the previously defined `get_valid_user_types()` function to make sure the values in the User Type column can be mapped to the appropriate license id values.  If not, print out a list of valid options
* use the previously defined `is_user_type_available()` function to ensure there is a `user type` license available in the org

In [ ]:
for idx, row in intern_data.iterrows():
    try:
        if not row["User Type"] in get_valid_user_types():
            print(f"User Type value must be in\n"
                  f"\t{[lic['name'] for lic in gis.users.license_types]}")
        if not is_user_type_available(row["User Type"]):
            print(f"\n{row['User Type']} not available. Check with Org Administrator.")
        new_user = gis.users.create(username=row["Username"],
                                    password=row["Password"],
                                    firstname=row["First Name"],
                                    lastname=row["Last Name"],
                                    email=row["Email"],
                                    user_type=row["User Type"],
                                    role=row["Role"])
        print(f"...created {new_user.username}")
        disable_pt = dt.datetime.fromtimestamp(new_user.created/1000) 
    except Exception as e:
        print(f"\nUnable to create user")
        print(f"{sys.exc_info()[0]}")
        print(f"{sys.exc_info()[1]}")
        print(f"\n")

## Disable users based on create date

In [ ]:
# datetime object for one minute after last user was created
disable_pt = disable_pt + relativedelta(minutes=1)

# datetime object in milliseconds
disable_pt_qy = int(disable_pt.timestamp()*1000)

In [ ]:
# 3 months from now
three_mos_dt = relativedelta(days=90)
three_mos_future = disable_pt + three_mos_dt

# dateime formats for searching
three_mos_disable_qy = int(three_mos_future.timestamp()*1000)

In [ ]:
for user in list_of_users:
    if int(((three_mos_disable_qy - user.created)/1000)/86400) > 90:
        user.disable()
        print(f"...{user.username:<22} disabled")
    else:
        user.enable()
        print(f"...{user.username:<22} remains active")

#### Demo: Disable the users we just created

In [ ]:
one_hr_ago = disable_pt - relativedelta(minutes=60)
one_hr_qy = int(one_hr_ago.timestamp()*1000)

recent_users = [u
               for u in gis.users.search(f"created:[{one_hr_qy} TO {disable_pt_qy}]")]

recent_users

In [ ]:
for user recent_users:
    if int(((one_hr_qy - user.created)/1000)/3600) < 1: # if user was created within last 24hrs
        print(f"...{user.username:<22} disabled")
    else:
        user.enable()
        print(f"...{user.username:<22} remains active")

## Revoke licenses and reassign content before deleting users

In [ ]:
lic_mgr = gis.admin.license

for user in recent_users:
    user.disable()
    try:
        user.delete()
        print(f"...deleted {user.username}")
    except Exception as e:
        if user.items() or [i for f in user.folders 
                            for i in user.items(folder=f)] or gis.groups.search(f"owner:{user.username}"):
            user.reassign_to(gis.users.me.username)
            print(f"...reassigning items and/or groups from {user.username}")
        add_on_provisions = user.provisions
        if add_on_provisions:
            for provision in add_on_provisions:
                provision_license = lic_mgr.get(provision.title)
                provision_license.revoke(user.username, entitlements="*")
        user_bundles = user.bundles
        if user_bundles:
            for bundle in user_bundles:
                bundle.revoke(user)
        user.delete()
        print(f"...deleted {user.username}")
        print(f"\n")